Mean to visualize more information related to Planet's CA coverage. 

In [1]:
api_key = 

In [2]:
import os
import json 
import requests
from requests.auth import HTTPBasicAuth

In [3]:
session = requests.Session()
session.auth = (api_key, '')

In [9]:
# Sends an API request for current item types. 
result =  session.get('https://api.planet.com/data/v1/item-types/')

In [10]:
result_json = result.json()
for image_type in result_json['item_types']:
    print image_type['id'] + '    |    ' + image_type['display_name']


PSScene4Band    |    4-band PlanetScope scene
PSScene3Band    |    3-band PlanetScope scene
REScene    |    RapidEye basic scene
REOrthoTile    |    RapidEye ortho tile
Sentinel2L1C    |    Sentinel-2 tiles
PSOrthoTile    |    PlanetScope ortho tile
Landsat8L1G    |    Landsat 8 scenes


In [6]:
# Sends an API request for current item types. 
result =  session.get('https://api.planet.com/data/v1/asset-types/')

In [7]:
result_json = result.json()
for asset_type in result_json['asset_types']:
    print asset_type['id'] + '    |    ' + asset_type['display_name']

basic_analytic_rpc_nitf    |    RPC for basic analytic image in NITF format
basic_analytic_dn_xml_nitf    |    Basic analytic digital number image metadata in NITF format
basic_analytic_xml    |    Basic analytic image metadata
basic_analytic_nitf    |    Basic analytic image in NITF format
analytic_bqa    |    Band QA
analytic_dn    |    Analytic digital number image
basic_analytic_b2    |    Green Band (Band2) of the Analytic Basic scene
basic_analytic_b3    |    Red Band (Band3) of the Analytic Basic scene
basic_analytic_b5_nitf    |    Near Infrared Band (Band5) of the Analytic Basic scene in the NITF format
basic_analytic_b1_nitf    |    Blue Band (Band1) of the Analytic Basic scene in the NITF format
analytic_b5    |    Band 5
metadata_txt    |    Metadata in a text format
analytic_b11    |    Band 11
analytic_b10    |    Band 10
analytic_b12    |    Band 12
basic_analytic_b2_nitf    |    Green Band (Band2) of the Analytic Basic scene in the NITF format
browse    |    Basic Scene

In [4]:
# Bring in our california filter
from california_filter import california

In [7]:
# Builds the API request for the 4-band Planet images 
search_request = {
  "item_types": ["PSScene4Band"],
  "filter": california
}

# Actually submit the request  
result = session.post('https://api.planet.com/data/v1/quick-search',
            json=search_request)

# Write results to a file 
with open('results.json', 'w') as f:
    result_json = result.json()
    f.write(json.dumps(result_json, indent = 4))

In [5]:
# Builds the API request for the 4-band Planet images - a summary of daily image count (not important for my project)
search_request = {
    "interval": "day",
    "item_types": ["PSScene4Band"],
    "filter": california
}

# Actually submit the request  
result = session.post('https://api.planet.com/data/v1/stats',
            json=search_request) 

# Write results to a file 
with open('results.json', 'w') as f:
    result_json = result.json()
    f.write(json.dumps(result_json, indent = 4))

In [17]:
# Builds the API request for the 4-band Planet images - dealing with paginated results 
search_request = {
    "name": "ca_coverage",
    "item_types": ["PSScene4Band"],
    "filter": california
}

# Submit request to create a search and save off id  
result = session.post('https://api.planet.com/data/v1/searches/',
            json=search_request)

search_id = result.json()["id"]    


# Create an empty array to hold ALL of the results
feature_list = []
image_metadata = {'type' : 'FeatureCollection',
                  'features': feature_list}

# Adds each result to our feature list
def handle_page(page):
    for item in page["features"]:
        feature_list.append(item)

# Recursively fetches and processes each page of search results 
def fetch_page(search_url):
    page = session.get(search_url).json()
    handle_page(page)
    next_url = page["_links"].get("_next")
    if next_url:
        fetch_page(next_url)

# Create URL for first page of results 
first_page = ("https://api.planet.com/data/v1/searches/{}" +
                "/results?_page_size={}").format(search_id, 100)

# Kick off API requests
fetch_page(first_page)

# Write results to a file 
with open('results.json', 'w') as f:
    result_json = result.json()
    f.write(json.dumps(result_json, indent = 4))

In [16]:
# Write results to a file 
with open('results.json', 'w') as f:
    result_json = result.json()
    f.write(json.dumps(result_json, indent = 4))


In [22]:
print len(feature_list)
print len(image_metadata['features'])

1201
1201


In [6]:
print json.dumps(california['config'][1], indent = 4)

{
    "field_name": "acquired", 
    "config": {
        "gte": "2017-04-15T12:00:00.000Z", 
        "lte": "2017-04-16T12:00:00.000Z"
    }, 
    "type": "DateRangeFilter"
}
